In [1]:
import requests
import pandas as pd
import numpy as np
import glob
import os
import re
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI


### Getting Master DB Data

In [2]:

# --- CONFIG ---
file_path = "./Master DB/Master_DB_oct22.xlsx"

# --- HELPER FUNCTIONS ---
def clean_uen(u: str) -> str | None:
    if pd.isna(u):
        return None
    return re.sub(r"[^A-Z0-9]", "", str(u).upper().strip())

def clean_text(text: str) -> str | None:
    if pd.isna(text):
        return None
    text = str(text).strip().upper()
    return None if text == "NAN" else text

def standardize_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Convert all column names to uppercase, replace non-alphanumeric with single underscore, remove trailing underscores."""
    new_cols = []
    for col in df.columns:
        col_std = re.sub(r"[^A-Z0-9]", "_", col.upper().strip())
        col_std = re.sub(r"_+", "_", col_std)  # Replace multiple underscores with single
        col_std = col_std.strip("_")  # Remove leading/trailing underscores
        new_cols.append(col_std)
    df.columns = new_cols
    return df

# --- LOAD DATA ---
master_db_df = pd.read_excel(file_path)

# --- SELECT RELEVANT COLUMNS ---
columns_to_keep = [
    "Company Registration Number (UEN)",
    "ACRA REGISTERED NAME",
    "Brand/Deal Name/Business Name",
    "Primary SSIC Code",
    "PIC NAME 1 Contact Number",
    "PIC 1 email address",
    "Website URL",
    "Parent Industry Type",
    "Sub Industry"
]
master_db_df = master_db_df[columns_to_keep].copy()

# --- STANDARDIZE COLUMN NAMES ---
master_db_df = standardize_columns(master_db_df)

# --- CLEANING & RENAME SPECIFIC COLUMNS ---
# Dynamically find the UEN column (first column containing 'UEN')
uen_col = [c for c in master_db_df.columns if "UEN" in c][0]
master_db_df["UEN"] = master_db_df[uen_col].apply(clean_uen)
master_db_df = master_db_df.drop(columns=[uen_col])

# Rename other columns consistently
rename_map = {
    "BRAND_DEAL_NAME_BUSINESS_NAME": "BRAND_NAME",
    "PRIMARY_SSIC_CODE": "SSIC_CODE",
    "ACRA_REGISTERED_NAME": "ACRA_REGISTERED_NAME"
}
master_db_df = master_db_df.rename(columns={k: v for k, v in rename_map.items() if k in master_db_df.columns})

# Clean text columns
for col in ["ACRA_REGISTERED_NAME", "BRAND_NAME"]:
    if col in master_db_df.columns:
        master_db_df[col] = master_db_df[col].apply(clean_text)

# Convert SSIC_CODE to integer if exists
if "SSIC_CODE" in master_db_df.columns:
    master_db_df["SSIC_CODE"] = master_db_df["SSIC_CODE"].astype("Int64")

# Keep only required columns if they exist
required_cols = ["UEN", "ACRA_REGISTERED_NAME", "BRAND_NAME", "SSIC_CODE"]
master_db_df = master_db_df[[c for c in required_cols if c in master_db_df.columns]]

# Filter out rows with missing or empty UEN
master_db_df = master_db_df[master_db_df["UEN"].notna() & (master_db_df["UEN"].str.strip() != "")]

master_db_df


,UEN,ACRA_REGISTERED_NAME,BRAND_NAME,SSIC_CODE
0,04799400B,AIK BEE TEXTILE CO,AIK BEE TEXTILE CO,46411
1,03376200K,SERANGOON GARDEN CLINIC AND DISPENSARY,GARDEN CLINIC,550263
2,06239600E,SALON DE BENZIMEN,SALON DE BENZIMEN,96021
3,06952000C,SU LAN LADIES FASHION,SU LAN LADIES FASHION,14103
4,10381600C,SIN HAI PRINTING SERVICE,SIN HAI PRINTING SERVICE,18113
...,...,...,...,...
7444,201734006N,MISTER MOBILE HOUGANG PTE. LTD.,MISTER MOBILE (HOUGANG),95120
7445,202210879W,MISTER MOBILE CHINATOWN PTE. LTD.,MISTER MOBILE (CHINATOWN),47411
7446,202205507G,MISTER MOBILE PTE. LTD.,MISTER MOBILE HQ,64202
7454,53473046M,BLOONIES,BLOONIES,47742


### Getting ACRA Data (Filter by Live, Live Company only & non relevant ssic code)

In [3]:
# -------------------------------------------------------------
# Folder containing your CSVs
# -------------------------------------------------------------
folder_path = "Acra_Data"

# Get all CSV file paths inside the folder
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

# Read and combine all CSVs
df = pd.concat((pd.read_csv(f) for f in csv_files), ignore_index=True)

# -------------------------------------------------------------
# Convert all column names to uppercase
# -------------------------------------------------------------
df.columns = df.columns.str.upper()


# -------------------------------------------------------------
# Select relevant columns (now in uppercase)
# -------------------------------------------------------------
acra_data = df[[
    "UEN",
    "ENTITY_NAME",
    "BUSINESS_CONSTITUTION_DESCRIPTION",
    "ENTITY_TYPE_DESCRIPTION",
    "ENTITY_STATUS_DESCRIPTION",
    "REGISTRATION_INCORPORATION_DATE",
    "PRIMARY_SSIC_CODE",
    "STREET_NAME",
    "POSTAL_CODE"
]].copy()

# -------------------------------------------------------------
# Convert to proper data types
# -------------------------------------------------------------
acra_data['UEN'] = acra_data['UEN'].astype('string')
acra_data['ENTITY_NAME'] = acra_data['ENTITY_NAME'].astype('string')
acra_data['BUSINESS_CONSTITUTION_DESCRIPTION'] = acra_data['BUSINESS_CONSTITUTION_DESCRIPTION'].astype('string')
acra_data['ENTITY_TYPE_DESCRIPTION'] = acra_data['ENTITY_TYPE_DESCRIPTION'].astype('string')
acra_data['ENTITY_STATUS_DESCRIPTION'] = acra_data['ENTITY_STATUS_DESCRIPTION'].astype('string')
acra_data['REGISTRATION_INCORPORATION_DATE'] = pd.to_datetime(acra_data['REGISTRATION_INCORPORATION_DATE'], errors='coerce')

# -------------------------------------------------------------
# Clean string columns — trim, remove extra spaces, uppercase
# -------------------------------------------------------------
for col in [
    'UEN',
    'ENTITY_NAME',
    'BUSINESS_CONSTITUTION_DESCRIPTION',
    'ENTITY_TYPE_DESCRIPTION',
    'ENTITY_STATUS_DESCRIPTION',
    'STREET_NAME',
    'POSTAL_CODE'
]:
    acra_data[col] = (
        acra_data[col]
        .fillna('')
        .str.strip()
        .str.replace(r'\s+', ' ', regex=True)
        .str.upper()
    )

# -------------------------------------------------------------
# Replace placeholders with NaN for standardization
# -------------------------------------------------------------
acra_data.replace(['NA', 'N/A', '-', ''], np.nan, inplace=True)

# -------------------------------------------------------------
# Convert registration date to dd-mm-yyyy string (optional)
# -------------------------------------------------------------
acra_data['REGISTRATION_INCORPORATION_DATE'] = acra_data['REGISTRATION_INCORPORATION_DATE'].dt.strftime('%d-%m-%Y')

# -------------------------------------------------------------
# Filter only live entities (LIVE COMPANY or LIVE)
# -------------------------------------------------------------
acra_data = acra_data[
    acra_data['ENTITY_STATUS_DESCRIPTION'].isin(['LIVE COMPANY', 'LIVE'])
].reset_index(drop=True)

# -------------------------------------------------------------
# Exclude specific PRIMARY_SSIC_CODE values
# -------------------------------------------------------------
exclude_codes = [
    46900, 47719, 47749, 47539, 47536, 56123,
    10711, 10712, 10719, 10732, 10733, 93209
]

acra_data = acra_data[~acra_data['PRIMARY_SSIC_CODE'].isin(exclude_codes)].reset_index(drop=True)

C:\Users\azlie\AppData\Local\Temp\ipykernel_1900\554762848.py:10: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(f) for f in csv_files), ignore_index=True)
C:\Users\azlie\AppData\Local\Temp\ipykernel_1900\554762848.py:10: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(f) for f in csv_files), ignore_index=True)
C:\Users\azlie\AppData\Local\Temp\ipykernel_1900\554762848.py:10: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(f) for f in csv_files), ignore_index=True)
C:\Users\azlie\AppData\Local\Temp\ipykernel_1900\554762848.py:10: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(f) for f in csv_files), ignore_index=True)
C:\Users\azlie\AppData\Local\Temp\ipyker

In [4]:
acra_data

,UEN,ENTITY_NAME,BUSINESS_CONSTITUTION_DESCRIPTION,ENTITY_TYPE_DESCRIPTION,ENTITY_STATUS_DESCRIPTION,REGISTRATION_INCORPORATION_DATE,PRIMARY_SSIC_CODE,STREET_NAME,POSTAL_CODE
0,00182000A,AIK SENG HENG,PARTNERSHIP,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,07-02-1975,46302,FISHERY PORT ROAD,619742
1,00233500W,ASIA STORE,PARTNERSHIP,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,28-10-1974,46411,SIMS AVENUE,387509
2,00733000J,AIK CHE HIONG,PARTNERSHIP,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,02-11-1974,32909,ANG MO KIO INDUSTRIAL PARK 2A,568049
3,00927000X,A WALIMOHAMED BROS,PARTNERSHIP,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,12-11-1974,46411,JELLICOE ROAD,208767
4,01173000E,ANG TECK MOH DEPARTMENT STORE,PARTNERSHIP,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,30-10-1974,47711,WOODLANDS STREET 12,738623
...,...,...,...,...,...,...,...,...,...
537323,T25LL0518K,ZEUS BARBERS LLP,<NA>,LIMITED LIABILITY PARTNERSHIP,LIVE,16-05-2025,96021,KELANTAN LANE,200031
537324,T25LL0858C,ZENSE SPACE LLP,<NA>,LIMITED LIABILITY PARTNERSHIP,LIVE,01-08-2025,43301,YISHUN INDUSTRIAL STREET 1,768161
537325,T25LL0870A,ZIQZEQ PROCUREMENT LLP,<NA>,LIMITED LIABILITY PARTNERSHIP,LIVE,04-08-2025,70209,SIN MING LANE,573969
537326,T25LL1049B,ZHONG XIN TRAVEL LLP,<NA>,LIMITED LIABILITY PARTNERSHIP,LIVE,08-09-2025,79102,JALAN BAHAGIA,320034


### Getting SSIC Industry code (some ssic codes are not present because of merging, so there will be null issues)

In [5]:
# --- CONFIG ---
file_path = "./SSIC_Code/mapped_ssic_code.xlsx"

# --- LOAD DATA ---
mapped_ssic_code = pd.read_excel(file_path)

# --- STANDARDIZE COLUMN NAMES ---
# Uppercase, strip spaces, replace spaces with underscores
mapped_ssic_code.columns = (
    mapped_ssic_code.columns
    .str.strip()
    .str.upper()
    .str.replace(" ", "_")
)

# --- KEEP ONLY DESIRED COLUMNS ---
columns_to_keep = ["PARENT_INDUSTRY", "INDUSTRY_TYPE", "SUB_INDUSTRY", "SSIC_CODES", "DESCRIPTION"]
mapped_ssic_code = mapped_ssic_code[columns_to_keep].copy()

# --- CLEAN SSIC_CODES COLUMN ---
mapped_ssic_code["SSIC_CODES"] = (
    pd.to_numeric(mapped_ssic_code["SSIC_CODES"], errors="coerce")  # safely convert to numeric
    .fillna(0)
    .astype(int)
)

# --- CLEAN TEXT COLUMNS ---
text_cols = ["PARENT_INDUSTRY", "INDUSTRY_TYPE", "SUB_INDUSTRY", "DESCRIPTION"]
mapped_ssic_code[text_cols] = mapped_ssic_code[text_cols].apply(
    lambda col: col.astype(str).str.strip().str.title()
)

# --- REMOVE DUPLICATES & RESET INDEX ---
mapped_ssic_code = mapped_ssic_code.drop_duplicates().reset_index(drop=True)

mapped_ssic_code.head()


,PARENT_INDUSTRY,INDUSTRY_TYPE,SUB_INDUSTRY,SSIC_CODES,DESCRIPTION
0,Retail,Retail,Fashion & Apparel,47711,Retail Sale Of Clothing For Adults
1,Retail,Retail,Fashion & Apparel,47712,Retail Sale Of Children And Infants' Clothing
2,Retail,Retail,Fashion & Apparel,47715,Retail Sale Of Sewing And Clothing Accessories
3,Retail,Retail,Fashion & Apparel,47719,"Retail Sale Of Clothing, Footwear And Leather ..."
4,Retail,Retail,Fashion & Apparel,47510,Retail Sale Of Textiles


### Merge ACRA data with SSIC code industry type

In [6]:
# Convert PRIMARY_SSIC_CODE to int
acra_data["PRIMARY_SSIC_CODE"] = (
    pd.to_numeric(acra_data["PRIMARY_SSIC_CODE"], errors="coerce")
    .fillna(0)
    .astype(int)
)

# Merge based on SSIC code
acra_data_filtered = acra_data.merge(
    mapped_ssic_code,
    how="left",
    left_on="PRIMARY_SSIC_CODE",
    right_on="SSIC_CODES"
)

# Optional: drop the duplicate 'SSIC CODES' column (keep only PRIMARY_SSIC_CODE)
acra_data_filtered = acra_data_filtered.drop(columns=["SSIC_CODES"], errors="ignore")


In [7]:
acra_data_filtered

,UEN,ENTITY_NAME,BUSINESS_CONSTITUTION_DESCRIPTION,ENTITY_TYPE_DESCRIPTION,ENTITY_STATUS_DESCRIPTION,REGISTRATION_INCORPORATION_DATE,PRIMARY_SSIC_CODE,STREET_NAME,POSTAL_CODE,PARENT_INDUSTRY,INDUSTRY_TYPE,SUB_INDUSTRY,DESCRIPTION
0,00182000A,AIK SENG HENG,PARTNERSHIP,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,07-02-1975,46302,FISHERY PORT ROAD,619742,Others,Wholesale Trade,"Food, Beverages & Tobacco","Wholesale Of Livestock, Meat, Poultry, Eggs An..."
1,00233500W,ASIA STORE,PARTNERSHIP,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,28-10-1974,46411,SIMS AVENUE,387509,Others,Wholesale Trade,Household Goods,Wholesale Of Textiles And Leathers
2,00733000J,AIK CHE HIONG,PARTNERSHIP,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,02-11-1974,32909,ANG MO KIO INDUSTRIAL PARK 2A,568049,Others,Manufacturing,Other Specialised Manufacturing & Distribution,Other Manufacturing Industries N.E.C.
3,00927000X,A WALIMOHAMED BROS,PARTNERSHIP,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,12-11-1974,46411,JELLICOE ROAD,208767,Others,Wholesale Trade,Household Goods,Wholesale Of Textiles And Leathers
4,01173000E,ANG TECK MOH DEPARTMENT STORE,PARTNERSHIP,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,30-10-1974,47711,WOODLANDS STREET 12,738623,Retail,Retail,Fashion & Apparel,Retail Sale Of Clothing For Adults
...,...,...,...,...,...,...,...,...,...,...,...,...,...
537323,T25LL0518K,ZEUS BARBERS LLP,<NA>,LIMITED LIABILITY PARTNERSHIP,LIVE,16-05-2025,96021,KELANTAN LANE,200031,Services,Services,Hair Salons & Barbershops,Hairdressing Salons/Shops
537324,T25LL0858C,ZENSE SPACE LLP,<NA>,LIMITED LIABILITY PARTNERSHIP,LIVE,01-08-2025,43301,YISHUN INDUSTRIAL STREET 1,768161,Others,Built Environment & Infrastructure,Construction,Renovation Contractors
537325,T25LL0870A,ZIQZEQ PROCUREMENT LLP,<NA>,LIMITED LIABILITY PARTNERSHIP,LIVE,04-08-2025,70209,SIN MING LANE,573969,Others,"Finance, Legal & Real Estate","Legal, Accounting & Consultancy Activities",Management Consultancy Services N.E.C.
537326,T25LL1049B,ZHONG XIN TRAVEL LLP,<NA>,LIMITED LIABILITY PARTNERSHIP,LIVE,08-09-2025,79102,JALAN BAHAGIA,320034,Others,"Tourism, Agency",Travel Agencies & Tour Operators,Travel Agencies And Tour Operators (Mainly Out...


### FIlter Acra data with Master DB to get list of companies havent been researched  by MR

In [8]:

# Ensure both UEN columns are strings for accurate matching
acra_data_filtered['UEN'] = acra_data_filtered['UEN'].astype(str).str.strip().str.upper()
master_db_df['UEN'] = master_db_df['UEN'].astype(str).str.strip().str.upper()

# Filter out rows in acra_data_filtered whose UEN is already in master_db_df
acra_data_filtered = acra_data_filtered[~acra_data_filtered['UEN'].isin(master_db_df['UEN'])]

acra_data_filtered.shape

(533824, 13)

### Filter by Wholesale Industry

In [9]:
# wholesale data
ssic_codes = [
    "46", "461", "4610", "46100", "462", "4621", "46211", "46212", "46213", "46219",
    "4622", "46221", "46222", "46223", "46224", "46225", "46229", "463", "4630", "46301",
    "46302", "46303", "46304", "46305", "46306", "46307", "46308", "46309", "464", "4641",
    "46411", "46412", "46413", "46414", "46415", "46416", "4642", "46421", "46422", "46423",
    "46424", "46429", "4643", "46431", "46432", "46433", "46434", "46435", "46436", "46439",
    "4644", "46441", "46442", "46443", "46444", "46445", "46449", "4645", "46451", "46452",
    "46453", "46459", "4646", "46461", "46462", "4647", "46471", "46472", "46473", "46474",
    "46479", "4649", "46491", "46492", "46499", "465", "4651", "46511", "46512", "46513",
    "46514", "4652", "46521", "46522", "46523", "4653", "46530", "4654", "46541", "46542",
    "46543", "46544", "46549", "4655", "46551", "46552", "46559", "4656", "46561", "46562",
    "46563", "4659", "46591", "46592", "46593", "46594", "46595", "46599", "466", "4661",
    "46610", "4662", "46620", "4663", "46631", "46632", "46633", "46634", "46635", "46639",
    "4664", "46641", "46642", "46643", "46649", "4665", "46651", "46659", "4666", "46661",
    "46662", "469", "4690", "46900"
]


acra_data_filtered_wholesale = acra_data_filtered[
    (
        (acra_data_filtered["ENTITY_STATUS_DESCRIPTION"].str.lower() == "live") |
        (acra_data_filtered["ENTITY_STATUS_DESCRIPTION"].str.lower() == "live company")
    )
    &
    (acra_data_filtered["PRIMARY_SSIC_CODE"].astype(str).isin(ssic_codes))
]


acra_data_filtered_wholesale

,UEN,ENTITY_NAME,BUSINESS_CONSTITUTION_DESCRIPTION,ENTITY_TYPE_DESCRIPTION,ENTITY_STATUS_DESCRIPTION,REGISTRATION_INCORPORATION_DATE,PRIMARY_SSIC_CODE,STREET_NAME,POSTAL_CODE,PARENT_INDUSTRY,INDUSTRY_TYPE,SUB_INDUSTRY,DESCRIPTION
0,00182000A,AIK SENG HENG,PARTNERSHIP,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,07-02-1975,46302,FISHERY PORT ROAD,619742,Others,Wholesale Trade,"Food, Beverages & Tobacco","Wholesale Of Livestock, Meat, Poultry, Eggs An..."
1,00233500W,ASIA STORE,PARTNERSHIP,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,28-10-1974,46411,SIMS AVENUE,387509,Others,Wholesale Trade,Household Goods,Wholesale Of Textiles And Leathers
3,00927000X,A WALIMOHAMED BROS,PARTNERSHIP,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,12-11-1974,46411,JELLICOE ROAD,208767,Others,Wholesale Trade,Household Goods,Wholesale Of Textiles And Leathers
12,04129500E,AIK HOE & CO,SOLE-PROPRIETOR,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,23-01-1975,46551,KELANTAN ROAD,200028,Others,Wholesale Trade,"Machinery, Equipment & Supplies",Wholesale Of Marine Equipment And Accessories
14,04545400X,AIK HUAT AND COMPANY,PARTNERSHIP,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,17-01-1975,46441,KAKI BUKIT AVENUE 1,417943,Others,Wholesale Trade,Household Goods,Wholesale Of Sporting Goods And Equipment
...,...,...,...,...,...,...,...,...,...,...,...,...,...
537268,T17LP0162L,ZYA HOLDINGS LIMITED PARTNERSHIP,<NA>,LIMITED PARTNERSHIP,LIVE,21-10-2017,46100,NATHAN ROAD,248728,Others,Wholesale Trade,Other Specialised Wholesale,Wholesale On A Fee Or Commission Basis
537298,T22LL0564C,ZEN ENGINEERING & TRADING LLP,<NA>,LIMITED LIABILITY PARTNERSHIP,LIVE,31-05-2022,46543,TOH GUAN ROAD EAST,608586,Others,Wholesale Trade,"Machinery, Equipment & Supplies","Wholesale Of Lifts, Escalators And Industrial ..."
537302,T23LL0056G,ZECRYNE LLP,<NA>,LIMITED LIABILITY PARTNERSHIP,LIVE,13-01-2023,46301,BUKIT BATOK STREET 25,658881,Others,Wholesale Trade,"Food, Beverages & Tobacco",Wholesale Of Fruits And Vegetables
537313,T24LL0528K,ZOHMH LIMITED LIABILITY PARTNERSHIP,<NA>,LIMITED LIABILITY PARTNERSHIP,LIVE,07-05-2024,46303,WOODLANDS AVENUE 4,730844,Others,Wholesale Trade,"Food, Beverages & Tobacco",Wholesale Of A General Line Of Groceries


### Web Scrapping 

In [10]:
# Ensure the column is in datetime format
acra_data_filtered_wholesale["REGISTRATION_INCORPORATION_DATE"] = pd.to_datetime(
    acra_data_filtered_wholesale["REGISTRATION_INCORPORATION_DATE"], errors="coerce"
)

# Filter for companies registered after 2015-01-01
filtered = acra_data_filtered_wholesale[
    acra_data_filtered_wholesale["REGISTRATION_INCORPORATION_DATE"] > "2015-01-01"
]

# Sample 10 rows
acra_data_filtered_wholesale_10 = filtered.sample(n=100, random_state=42)

acra_data_filtered_wholesale_10

C:\Users\azlie\AppData\Local\Temp\ipykernel_1900\3023573331.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acra_data_filtered_wholesale["REGISTRATION_INCORPORATION_DATE"] = pd.to_datetime(


,UEN,ENTITY_NAME,BUSINESS_CONSTITUTION_DESCRIPTION,ENTITY_TYPE_DESCRIPTION,ENTITY_STATUS_DESCRIPTION,REGISTRATION_INCORPORATION_DATE,PRIMARY_SSIC_CODE,STREET_NAME,POSTAL_CODE,PARENT_INDUSTRY,INDUSTRY_TYPE,SUB_INDUSTRY,DESCRIPTION
55648,201819442H,BELL 1 SINGAPORE PTE. LTD.,<NA>,LOCAL COMPANY,LIVE COMPANY,2018-07-06,46610,SOON LEE STREET,627608,Others,Wholesale Trade,Other Specialised Wholesale,Wholesale Of Fuels And Related Products
324873,202534677Z,NUU&NII PTE. LTD.,<NA>,LOCAL COMPANY,LIVE COMPANY,2025-08-08,46413,WOODLANDS RISE,731785,Others,Wholesale Trade,Household Goods,Wholesale Of Children And Infants' Clothing
86032,201925464M,CLOUD FRONTIER ASIA PTE. LTD.,<NA>,LOCAL COMPANY,LIVE COMPANY,2019-05-08,46512,WOODLANDS CLOSE,737854,Others,Wholesale Trade,"Machinery, Equipment & Supplies",Wholesale Of Computer Software (Except Games A...
175697,202108485W,GOLDEN GARLAND COMMUNICATION PTE. LTD.,<NA>,LOCAL COMPANY,LIVE COMPANY,2021-10-03,46523,NORTH BRIDGE ROAD,179094,Others,Wholesale Trade,"Machinery, Equipment & Supplies",Wholesale Of Handphones And Related Telecommun...
522415,202519922K,XINYUAN SHOES PTE. LTD.,<NA>,LOCAL COMPANY,LIVE COMPANY,2025-08-05,46414,ANG MO KIO STREET 21,562260,Others,Wholesale Trade,Household Goods,Wholesale Of Footwear
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16252,201827062C,APOLLO HEALTHCARE PTE. LTD.,<NA>,LOCAL COMPANY,LIVE COMPANY,2018-08-08,46461,SIN MING LANE,NaN,Others,Hospital,Pharmacies & Health Stores,Wholesale Of Medicinal And Pharmaceutical Prod...
430269,202408802M,SUCCESS AGRITRADE PTE. LTD.,<NA>,LOCAL COMPANY,LIVE COMPANY,2024-06-03,46219,UBI AVENUE 3,408868,Others,Wholesale Trade,Agricultural Raw Materials & Live Animals,Wholesale Of Agricultural Raw Materials And Li...
417865,202019332W,SQ BULLION PTE. LTD.,<NA>,LOCAL COMPANY,LIVE COMPANY,2020-07-07,46421,PAYA LEBAR ROAD,409030,Others,Wholesale Trade,Household Goods,Wholesale Of Jewellery Made From Precious Meta...
134805,202010339G,ETERNAL AEROSPACE PTE. LTD.,<NA>,LOCAL COMPANY,LIVE COMPANY,2020-01-04,46552,FLORA DRIVE,506868,Others,Wholesale Trade,"Machinery, Equipment & Supplies",Wholesale Of Aircraft Equipment And Supplies


In [11]:
acra_data_filtered_wholesale_10.shape

(100, 13)

In [12]:
import time
import pandas as pd
from apify_client import ApifyClient
from bs4 import BeautifulSoup
import re
import json

client = ApifyClient("apify_api_XrMF5yGkUFSpPwIfmCSCzoALB2sGxR2JtFnt")

SOCIAL_MEDIA_DOMAINS = [
    "facebook.com", "linkedin.com", "instagram.com", "youtube.com",
    "tiktok.com", "twitter.com", "x.com", "pinterest.com"
]

all_results = []

for i, row in acra_data_filtered_wholesale_10.iterrows():
    uen = row["UEN"]
    print(f"\n🔎 Processing {uen} ({i+1}/{len(acra_data_filtered_wholesale_10)})")

    # Build pageFunction as a *raw string*, no f-string injection
    page_function = f"""
    async function pageFunction(context) {{
        const {{ page, log, request }} = context;
        const uen = "{uen}";
        log.info("Visiting RecordOwl for UEN: " + uen);

        try {{
            await page.waitForSelector("input[placeholder='Search company name, industry, or address']", {{ timeout: 25000 }});
            const input = await page.$("input[placeholder='Search company name, industry, or address']");
            await input.click({{ clickCount: 3 }});
            await input.type(uen, {{ delay: 100 }});

            await Promise.all([
                page.waitForNavigation({{ waitUntil: 'networkidle2', timeout: 60000 }}).catch(() => null),
                page.click("button[type='submit']")
            ]);

            await page.waitForSelector("a[href*='/company/']", {{ timeout: 40000 }});
            const companyLink = await page.$$eval("a[href*='/company/']", (links, uen) => {{
                for (const a of links) {{
                    if (a.innerText.includes(uen) || a.href.includes(uen)) return a.href;
                }}
                return links.length > 0 ? links[0].href : null;
            }}, uen);

            if (!companyLink) return {{ status: 'not_found', uen }};

            if (page.url() !== companyLink) {{
                await page.goto(companyLink, {{ waitUntil: 'networkidle2', timeout: 60000 }});
            }}

            await new Promise(r => setTimeout(r, 2000));
            const html_content = await page.content();
            const title = await page.title();
            const url = page.url();

            return {{ status: 'success', uen, url, title, html_content }};
        }} catch (err) {{
            return {{ status: 'error', uen, error: err.message }};
        }}
    }}
    """

    run_input = {
        "startUrls": [{"url": "https://recordowl.com/"}],
        "useChrome": True,
        "headless": True,
        "stealth": True,
        "pageFunction": page_function,
    }

    try:
        run = client.actor("apify/puppeteer-scraper").call(
            run_input=run_input,
            memory_mbytes=2048,
            timeout_secs=300
        )
    except Exception as e:
        print(f"❌ Apify call failed for {uen}: {e}")
        continue

    if not run or "defaultDatasetId" not in run:
        print(f"⚠️ No valid dataset returned for {uen}")
        continue

    scraped_html, record_owl_url = None, None
    for item in client.dataset(run["defaultDatasetId"]).iterate_items():
        if item.get("status") == "success":
            scraped_html = item.get("html_content", "")
            record_owl_url = item.get("url")
        elif item.get("status") == "not_found":
            print(f"⚠️ Company not found for UEN {uen}")
        else:
            print(f"❌ Error for {uen}: {item.get('error')}")

    if not scraped_html:
        time.sleep(5)
        continue

    # ----- parse as before -----
    soup = BeautifulSoup(scraped_html, "html.parser")
    parent = soup.select_one("div.max-w-7xl.mx-auto.lg\\:py-6.sm\\:px-6.lg\\:px-8")

    emails, phones, website = [], [], None
    facebook_links, linkedin_links, instagram_links, tiktok_links = [], [], [], []

    if parent:
        for a in parent.select("a[href^=mailto]"):
            email = a.get("href").replace("mailto:", "").strip()
            if email not in emails:
                emails.append(email)

        for dt in parent.select("dt"):
            if "contact number" in dt.get_text(strip=True).lower():
                dd = dt.find_next_sibling("dd")
                if dd:
                    number_text = dd.get_text(" ", strip=True)
                    for match in re.findall(r"(?:\+65\s*|65)?(?:\d\s*){8,}", number_text):
                        p_clean = re.sub(r"\D", "", match)
                        if not p_clean.startswith("+65"):
                            p_clean = "+65" + p_clean[-8:]
                        phones.append(p_clean)

        valid_websites = []
        for a in parent.select("a[href^=http]"):
            href = a.get("href").strip()
            if not any(domain in href.lower() for domain in SOCIAL_MEDIA_DOMAINS):
                if not any(skip in href for skip in ["recordowl", "apify.com"]):
                    if (".com" in href or ".sg" in href or ".net" in href or ".org" in href):
                        valid_websites.append(href)
        website = valid_websites[0] if valid_websites else None

    for a in soup.find_all("a", href=True):
        href = a["href"].lower()
        if "facebook.com" in href:
            facebook_links.append(href)
        elif "linkedin.com" in href:
            linkedin_links.append(href)
        elif "instagram.com" in href:
            instagram_links.append(href)
        elif "tiktok.com" in href:
            tiktok_links.append(href)

    all_results.append({
        "UEN": uen,
        "Emails": emails or None,
        "Phones": phones or None,
        "Website": website or None,
        "Facebook": list(set(facebook_links)) or None,
        "LinkedIn": list(set(linkedin_links)) or None,
        "Instagram": list(set(instagram_links)) or None,
        "TikTok": list(set(tiktok_links)) or None,
        "RecordOwl_Link": record_owl_url or None,
    })

    time.sleep(8 + i % 3)

recordowl_results_df = pd.DataFrame(all_results)
print("\n✅ Scraping complete!")
print(recordowl_results_df.head())



🔎 Processing 201819442H (55649/10)


[apify.puppeteer-scraper runId:zg6sFYuRth42cWVc3] -> Status: RUNNING, Message: 
[apify.puppeteer-scraper runId:zg6sFYuRth42cWVc3] -> 2025-10-31T09:09:25.529Z ACTOR: Pulling container image of build g6G5r98rF5fM6ecm3 from registry.
[apify.puppeteer-scraper runId:zg6sFYuRth42cWVc3] -> 2025-10-31T09:09:25.532Z ACTOR: Creating container.
[apify.puppeteer-scraper runId:zg6sFYuRth42cWVc3] -> 2025-10-31T09:09:25.698Z ACTOR: Starting container.
[apify.puppeteer-scraper runId:zg6sFYuRth42cWVc3] -> 2025-10-31T09:09:26.057Z Will run command: xvfb-run -a -s "-ac -screen 0 1920x1080x24+32 -nolisten tcp" /bin/sh -c ./start_xvfb_and_run_cmd.sh && npm run start:prod --silent
[apify.puppeteer-scraper runId:zg6sFYuRth42cWVc3] -> 2025-10-31T09:09:27.180Z INFO  System info {"apifyVersion":"3.4.4","apifyClientVersion":"2.16.0","crawleeVersion":"3.14.1","osType":"Linux","nodeVersion":"v22.19.0"}
[apify.puppeteer-scraper runId:zg6sFYuRth42cWVc3] -> 2025-10-31T09:09:28.097Z INFO  Configuring Puppeteer Scraper


🔎 Processing 202534677Z (324874/10)


[apify.puppeteer-scraper runId:XuK4OIIQAYRlN2afx] -> Status: RUNNING, Message: 
[apify.puppeteer-scraper runId:XuK4OIIQAYRlN2afx] -> 2025-10-31T09:10:41.929Z ACTOR: Pulling container image of build g6G5r98rF5fM6ecm3 from registry.
[apify.puppeteer-scraper runId:XuK4OIIQAYRlN2afx] -> 2025-10-31T09:10:41.942Z ACTOR: Creating container.
[apify.puppeteer-scraper runId:XuK4OIIQAYRlN2afx] -> 2025-10-31T09:10:41.989Z ACTOR: Starting container.
[apify.puppeteer-scraper runId:XuK4OIIQAYRlN2afx] -> 2025-10-31T09:10:42.205Z Will run command: xvfb-run -a -s "-ac -screen 0 1920x1080x24+32 -nolisten tcp" /bin/sh -c ./start_xvfb_and_run_cmd.sh && npm run start:prod --silent
[apify.puppeteer-scraper runId:XuK4OIIQAYRlN2afx] -> 2025-10-31T09:10:43.712Z INFO  System info {"apifyVersion":"3.4.4","apifyClientVersion":"2.16.0","crawleeVersion":"3.14.1","osType":"Linux","nodeVersion":"v22.19.0"}
[apify.puppeteer-scraper runId:XuK4OIIQAYRlN2afx] -> 2025-10-31T09:10:44.008Z INFO  Configuring Puppeteer Scraper


🔎 Processing 201925464M (86033/10)


...


🔎 Processing 202207198G (297250/10)


[apify.puppeteer-scraper runId:Dk3dg1x6oGZR9XWyF] -> Status: RUNNING, Message: 
[apify.puppeteer-scraper runId:Dk3dg1x6oGZR9XWyF] -> 2025-10-31T09:19:42.662Z ACTOR: Pulling container image of build g6G5r98rF5fM6ecm3 from registry.
[apify.puppeteer-scraper runId:Dk3dg1x6oGZR9XWyF] -> 2025-10-31T09:19:42.664Z ACTOR: Creating container.
[apify.puppeteer-scraper runId:Dk3dg1x6oGZR9XWyF] -> 2025-10-31T09:19:42.809Z ACTOR: Starting container.
[apify.puppeteer-scraper runId:Dk3dg1x6oGZR9XWyF] -> 2025-10-31T09:19:43.051Z Will run command: xvfb-run -a -s "-ac -screen 0 1920x1080x24+32 -nolisten tcp" /bin/sh -c ./start_xvfb_and_run_cmd.sh && npm run start:prod --silent
[apify.puppeteer-scraper runId:Dk3dg1x6oGZR9XWyF] -> 2025-10-31T09:19:44.710Z INFO  System info {"apifyVersion":"3.4.4","apifyClientVersion":"2.16.0","crawleeVersion":"3.14.1","osType":"Linux","nodeVersion":"v22.19.0"}
[apify.puppeteer-scraper runId:Dk3dg1x6oGZR9XWyF] -> 2025-10-31T09:19:45.062Z INFO  Configuring Puppeteer Scraper


🔎 Processing 202449443E (96239/10)


[apify.puppeteer-scraper runId:RHWKoDjl97clCEKCa] -> Status: RUNNING, Message: 
[apify.puppeteer-scraper runId:RHWKoDjl97clCEKCa] -> 2025-10-31T09:21:02.107Z ACTOR: Pulling container image of build g6G5r98rF5fM6ecm3 from registry.
[apify.puppeteer-scraper runId:RHWKoDjl97clCEKCa] -> 2025-10-31T09:21:02.109Z ACTOR: Creating container.
[apify.puppeteer-scraper runId:RHWKoDjl97clCEKCa] -> 2025-10-31T09:21:02.155Z ACTOR: Starting container.
[apify.puppeteer-scraper runId:RHWKoDjl97clCEKCa] -> 2025-10-31T09:21:02.359Z Will run command: xvfb-run -a -s "-ac -screen 0 1920x1080x24+32 -nolisten tcp" /bin/sh -c ./start_xvfb_and_run_cmd.sh && npm run start:prod --silent
[apify.puppeteer-scraper runId:RHWKoDjl97clCEKCa] -> 2025-10-31T09:21:03.193Z INFO  System info {"apifyVersion":"3.4.4","apifyClientVersion":"2.16.0","crawleeVersion":"3.14.1","osType":"Linux","nodeVersion":"v22.19.0"}
[apify.puppeteer-scraper runId:RHWKoDjl97clCEKCa] -> 2025-10-31T09:21:03.337Z INFO  Configuring Puppeteer Scraper


✅ Scraping complete!
          UEN                Emails         Phones             Website  \
0  201819442H  [enquiry@bell-1.com]  [+6562590183]  https://bell-1.com   
1  202534677Z                  None           None                None   
2  201925464M                  None           None                None   
3  202108485W                  None           None                None   
4  202519922K                  None           None                None   

                                     Facebook LinkedIn  \
0  [https://www.facebook.com/bell1lubricant/]     None   
1                                        None     None   
2                                        None     None   
3                                        None     None   
4                                        None     None   

                                     Instagram TikTok  \
0  [https://www.instagram.com/bell1lubricant/]   None   
1                                         None   None   
2            

In [18]:
recordowl_results_df

,UEN,Emails,Phones,Website,Facebook,LinkedIn,Instagram,TikTok,RecordOwl_Link
0,201819442H,[enquiry@bell-1.com],[+6562590183],https://bell-1.com,[https://www.facebook.com/bell1lubricant/],None,[https://www.instagram.com/bell1lubricant/],None,https://recordowl.com/company/bell-1-singapore...
1,202534677Z,None,None,None,None,None,None,None,https://recordowl.com/company/nuu-nii-pte-ltd
2,201925464M,None,None,None,None,None,None,None,https://recordowl.com/company/cloud-frontier-a...
3,202108485W,None,None,None,None,None,None,None,https://recordowl.com/company/golden-garland-c...
4,202519922K,None,None,None,None,None,None,None,https://recordowl.com/company/xinyuan-shoes-pt...
5,201914673H,[enquiry@promoter.com.sg],[+6567743127],None,None,None,None,None,https://recordowl.com/company/theseus-promoter...
6,202200566K,None,None,None,None,None,None,None,https://recordowl.com/company/asia-digital-all...
7,202519905W,None,None,None,None,None,None,None,https://recordowl.com/company/nine-purple-gold...
8,202207198G,None,[+6596402556],None,None,None,[https://www.instagram.com/myfit.sg/],None,https://recordowl.com/company/myfit-enterprise...
9,202449443E,None,None,https://cdi-sg.com/,None,None,None,None,https://recordowl.com/company/cdindustry-inter...


### Web Scrapping

In [14]:
# from apify_client import ApifyClient

# client = ApifyClient("apify_api_0HQ8fc5fw5T1aosdacxKQNQYVBAEwi3tXaJc")

# uen = "202333361N"
# scraped_html = None  # Will hold the HTML content

# run_input = {
#     "startUrls": [{"url": "https://recordowl.com/"}],
#     "useChrome": True,
#     "headless": True,
#     "stealth": True,
#     "pageFunction": f"""
#     async function pageFunction(context) {{
#         const {{ page, log, request }} = context;
#         const uen = "{uen}";
#         log.info("🌐 Visiting " + request.url);

#         try {{
#             // Wait for search input and type UEN
#             await page.waitForSelector("input[placeholder='Search company name, industry, or address']", {{ timeout: 30000 }});
#             const input = await page.$("input[placeholder='Search company name, industry, or address']");
#             await input.click({{ clickCount: 3 }});
#             await input.type(uen, {{ delay: 80 }});
#             log.info("✅ Typed UEN: " + uen);

#             // Click search button and wait for navigation or results
#             const [response] = await Promise.all([
#                 page.waitForNavigation({{ waitUntil: "networkidle2", timeout: 60000 }}).catch(() => null), // in case no full navigation
#                 page.click("button[type='submit']"),
#             ]);
#             log.info("🔍 Clicked search button");

#             // Wait for company link(s) to appear
#             await page.waitForSelector("a[href*='/company/']", {{ timeout: 40000 }});

#             // Get the first matching company link
#             const companyLink = await page.$$eval("a[href*='/company/']", (links, uen) => {{
#                 for (const a of links) {{
#                     if (a.innerText.includes(uen) || a.href.includes(uen)) return a.href;
#                 }}
#                 return links.length > 0 ? links[0].href : null;
#             }}, uen);

#             if (!companyLink) return {{ status: "not_found", uen }};

#             // Navigate to the company page (if different)
#             if (page.url() !== companyLink) {{
#                 await page.goto(companyLink, {{ waitUntil: "networkidle2", timeout: 60000 }});
#             }}

#             // Small wait to ensure content is fully loaded
#             await new Promise(resolve => setTimeout(resolve, 2000));

#             // Extract HTML
#             const html_content = await page.content();
#             const title = await page.title();
#             const url = page.url();

#             return {{ status: "success", uen, url, title, html_content }};
#         }} catch (err) {{
#             log.error("❌ Error: " + err.message);
#             return {{ status: "error", uen, error: err.message }};
#         }}
#     }}
#     """
# }

# # Run the scraper
# run = client.actor("apify/puppeteer-scraper").call(run_input=run_input)

# # Store HTML into variable
# for item in client.dataset(run["defaultDatasetId"]).iterate_items():
#     if item.get("status") == "success":
#         scraped_html = item.get("html_content", "")
#         record_owl_url = item.get("url")  
        
#         print("✅ Scraped successfully")
#         print("Title:", item.get("title"))
#         print("RecordOwl URL:", record_owl_url)
#     elif item.get("status") == "not_found":
#         print(f"⚠️ Company not found for UEN {uen}")
#     else:
#         print("❌ Error:", item.get("error"))

In [15]:
# import pandas as pd
# from bs4 import BeautifulSoup
# import re

# # scraped_html and record_owl_url come from the Apify scraping code above
# # record_owl_url = item.get("url") from the Apify result

# # --- PARSE HTML ---
# soup = BeautifulSoup(scraped_html, "html.parser")

# # --- TARGET MAIN CONTAINER ---
# parent = soup.select_one("div.max-w-7xl.mx-auto.lg\\:py-6.sm\\:px-6.lg\\:px-8")

# emails, phones, website = [], [], None
# facebook_links, linkedin_links, instagram_links, tiktok_links = [], [], [], []
# number_of_employees = None

# # -------------------------------------------------------------
# # Extract Contact Info (Emails, Phones, Website)
# # -------------------------------------------------------------
# if parent:
#     for a in parent.select("a[href^=mailto]"):
#         email = a.get("href").replace("mailto:", "").strip()
#         if email not in emails:
#             emails.append(email)

#     for dt in parent.select("dt"):
#         label = dt.get_text(strip=True).lower()
#         if "contact number" in label:
#             dd = dt.find_next_sibling("dd")
#             if dd:
#                 number_text = dd.get_text(" ", strip=True)
#                 phone_pattern = r"(?:\+65\s*|65)?(?:\d\s*){8,}"
#                 for match in re.findall(phone_pattern, number_text):
#                     p_clean = re.sub(r"\D", "", match)
#                     if p_clean.startswith("65") and not p_clean.startswith("+65"):
#                         p_clean = "+" + p_clean
#                     elif not p_clean.startswith("+65"):
#                         p_clean = "+65" + p_clean
#                     if p_clean not in phones:
#                         phones.append(p_clean)

#     for a in parent.select("a[href^=http]"):
#         href = a.get("href").strip()
#         if any(skip in href.lower() for skip in ["recordowl", "apify.com"]):
#             continue
#         if (".com" in href or ".sg" in href) and not href.startswith(("mailto:", "tel:")):
#             website = href
#             break

# # -------------------------------------------------------------
# # Extract Social Media Links
# # -------------------------------------------------------------
# for a in soup.find_all("a", href=True):
#     href = a["href"].strip().lower()
#     if "facebook.com" in href:
#         facebook_links.append(href)
#     elif "linkedin.com" in href:
#         linkedin_links.append(href)
#     elif "instagram.com" in href:
#         instagram_links.append(href)
#     elif "tiktok.com" in href:
#         tiktok_links.append(href)

# facebook_links = list(set(facebook_links))
# linkedin_links = list(set(linkedin_links))
# instagram_links = list(set(instagram_links))
# tiktok_links = list(set(tiktok_links))

# # -------------------------------------------------------------
# # Extract Number of Employees
# # -------------------------------------------------------------
# for li in soup.select("li"):
#     li_text = li.get_text(" ", strip=True).lower()
#     if "number of employees" in li_text:
#         p_tags = li.find_all("p")
#         for i, p in enumerate(p_tags):
#             if "new value" in p.get_text(strip=True).lower() and i + 1 < len(p_tags):
#                 number_of_employees = p_tags[i + 1].get_text(strip=True)
#                 break
#         if number_of_employees:
#             break

# # -------------------------------------------------------------
# # SAVE RESULTS TO DATAFRAME (use None for empty values)
# # -------------------------------------------------------------
# result_df = pd.DataFrame([{
#     "Emails": emails if emails else None,
#     "Phones": phones if phones else None,
#     "Website": website if website else None,
#     "Facebook": facebook_links if facebook_links else None,
#     "LinkedIn": linkedin_links if linkedin_links else None,
#     "Instagram": instagram_links if instagram_links else None,
#     "TikTok": tiktok_links if tiktok_links else None,
#     "Number_of_Employees": number_of_employees if number_of_employees else None,
#     "RecordOwl_Link": record_owl_url if record_owl_url else None,
# }])

# display(result_df)

In [16]:
# import httpx
# import asyncio

# # =====================================================
# # Validate Website (only if no phone number)
# # =====================================================
# async def check_url(url: str) -> bool:
#     """Return True if the URL is reachable (status < 400)."""
#     if not url:
#         return False
#     try:
#         async with httpx.AsyncClient(follow_redirects=True, timeout=5) as client:
#             response = await client.head(url)
#             return response.status_code < 400
#     except Exception:
#         return False


# async def validate_if_needed(df):
#     """Validate websites only if phone number is missing."""
#     for i, row in df.iterrows():
#         url = row.get("Website")
#         phone = row.get("Phones")

#         # Skip validation if phone exists
#         if phone:
#             df.at[i, "Website_Valid"] = None
#             continue

#         # Validate website if no phone
#         if url:
#             is_valid = await check_url(url)
#             df.at[i, "Website_Valid"] = "valid" if is_valid else "invalid"
#         else:
#             df.at[i, "Website_Valid"] = "invalid"

#     return df


# # =====================================================
# # Run async validation safely inside Jupyter
# # =====================================================
# result_df = await validate_if_needed(result_df)

# # =====================================================
# # Final output
# # =====================================================
# display(result_df)


### If contact number is invalid, then webscrapped website to get contact number

In [17]:
# import asyncio
# import os
# from apify_client import ApifyClient

# # --- Initialize Apify client ---
# APIFY_TOKEN = os.getenv("APIFY_TOKEN", "apify_api_0HQ8fc5fw5T1aosdacxKQNQYVBAEwi3tXaJc")
# client = ApifyClient(APIFY_TOKEN)

# # --- Async wrapper so you can run in Jupyter ---
# async def enrich_with_contact_info(df):
#     """Scrape contact info for rows where Website_Valid == 'valid' and Phones is empty."""
#     updated_df = df.copy()

#     for i, row in df.iterrows():
#         website = row.get("Website")
#         status = row.get("Website_Valid")
#         phone = row.get("Phones")

#         if not website or status != "valid" or phone:
#             continue  # Skip invalid or already complete rows

#         print(f"🔍 Scraping contact page for: {website}")

#         # --- Apify scraping run input ---
#         run_input = {
#             "startUrls": [{"url": website}],
#             "pageFunction": r"""
#                 async function pageFunction(context) {
#                     const $ = context.jQuery;
#                     const isContact = context.request.userData?.isContact || false;

#                     if (!isContact) {
#                         let contactUrl = null;
#                         $('a[href]').each((i, el) => {
#                             const href = $(el).attr('href').toLowerCase();
#                             if (href.includes('contact')) {
#                                 contactUrl = href.startsWith('http') ? href : window.location.origin + href;
#                                 return false;
#                             }
#                         });

#                         if (contactUrl) {
#                             await context.enqueueRequest({ url: contactUrl, userData: { isContact: true } });
#                             context.log.info(`Enqueued contact page: ${contactUrl}`);
#                         }
#                         return null;
#                     }

#                     function isVisible(el) {
#                         return el.offsetParent !== null;
#                     }

#                     let emails = $('a[href^="mailto"]').filter((i, el) => isVisible(el))
#                         .map((i, el) => $(el).attr('href').replace('mailto:', '').trim())
#                         .get();

#                     let phones = $('a[href^="tel"]').filter((i, el) => isVisible(el))
#                         .map((i, el) => $(el).attr('href').replace(/[^0-9]/g, ''))
#                         .get();

#                     emails = [...new Set(emails)];
#                     phones = [...new Set(phones)];

#                     return {
#                         contactUrl: context.request.url,
#                         emails: emails.length ? emails : [],
#                         phones: phones.length ? phones : []
#                     };
#                 }
#             """,
#             "injectJQuery": True,
#             "useChrome": True,
#             "headless": True,
#             "proxyConfiguration": {"useApifyProxy": True},
#         }

#         # --- Run the Apify scraper ---
#         try:
#             run = client.actor("moJRLRc85AitArpNN").call(run_input=run_input)
#             dataset = client.dataset(run["defaultDatasetId"])
#             results = list(dataset.iterate_items())
#             contact_results = [r for r in results if r and (r.get("emails") or r.get("phones"))]

#             if contact_results:
#                 scraped = contact_results[0]
#                 updated_df.at[i, "Emails"] = scraped.get("emails", None)
#                 updated_df.at[i, "Phones"] = scraped.get("phones", None)
#                 updated_df.at[i, "Contact_Page"] = scraped.get("contactUrl", None)
#                 print(f"✅ Found: {scraped.get('phones', [])} / {scraped.get('emails', [])}")
#             else:
#                 print("⚠️ No contact data found.")

#         except Exception as e:
#             print(f"❌ Error scraping {website}: {e}")

#     return updated_df


# # --- Run the scraper for valid websites ---
# result_df = await enrich_with_contact_info(result_df)

# # --- Display updated results ---
# display(result_df)
